In [17]:
import numpy as np

from sklearn import datasets, linear_model, svm
import matplotlib.pyplot as plt
import pandas as pd

from scipy import linalg

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sklearn.utils import check_array

from sklearn.utils._testing import assert_array_almost_equal
from sklearn.utils._testing import assert_array_equal
from sklearn.utils._testing import ignore_warnings
from sklearn.utils._testing import TempMemmap

from sklearn.decomposition import DictionaryLearning
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import SparseCoder
from sklearn.decomposition import dict_learning
from sklearn.decomposition import dict_learning_online
from sklearn.decomposition import sparse_encode

rng_global = np.random.RandomState(0)
n_samples, n_features = 10, 8
X = rng_global.randn(n_samples, n_features)
n_components = 12

def test_dict_learning_reconstruction():
    n_components = 12
    dico = DictionaryLearning(n_components, transform_algorithm='lasso_lars',
                              transform_alpha=0.001, random_state=0)
    code = dico.fit(X).transform(X)
    assert_array_almost_equal(np.dot(code, dico.components_), X, decimal=2)
test_dict_learning_reconstruction()

In [4]:
code, dict_, err = dict_learning(X, n_components = 12, alpha = .0001)
assert_array_almost_equal(np.dot(code, dict_), X, decimal=2)
code, dict_ = dict_learning_online(X, n_components = 12, alpha = .0001)
assert_array_almost_equal(np.dot(code, dict_), X, decimal=2)
print(code.shape)
print(dict_.shape)

(10, 12)
(12, 8)


In [6]:
from sklearn.decomposition import sparse_encode
code_ = sparse_encode(X, dict_, alpha = .00001)
print(code_.shape)

(10, 12)


In [7]:
new_X = np.dot(code_, dict_)
print(new_X.shape)

(10, 8)


In [8]:
assert_array_almost_equal(np.dot(code_, dict_), X, decimal=2)

In [14]:
from sklearn.decomposition._dict_learning_na import sparse_encode_na



In [51]:

def dict_learning_na(X, n_components=2, alpha=1, n_iter=100):
    
    code, S, dictionary = linalg.svd(X, full_matrices=False)
    dictionary = S[:, np.newaxis] * dictionary
    
    r = len(dictionary)
    if n_components <= r:  # True even if n_components=None
        code = code[:, :n_components]
        dictionary = dictionary[:n_components, :]
    else:
        code = np.c_[code, np.zeros((len(code), n_components - r))]
        dictionary = np.r_[dictionary,
                           np.zeros((n_components - r, dictionary.shape[1]))]

        
    print(dictionary.shape)
    print(code.shape)
    
    D = dictionary
    for x in X:   
        
        this_code = sparse_encode_na(x, D, alpha)
        C, B, e = update1(x, this_code, C, B, e, alpha)
        D = update_dict(C, B, e, D, code)
        e = update2(e, x, D, this_code)
        
    return code, D



In [52]:
dict_learning_na(X)

(2, 8)
(10, 2)


NameError: name 'update1' is not defined

In [45]:
D.shape

(8, 8)

In [43]:
## test

import numpy as np

from sklearn.decomposition._dict_learning_na import sparse_encode_na

from sklearn.utils._testing import assert_array_almost_equal
from sklearn.utils._testing import assert_array_equal


def test_shape_encode():
    
    this_x = X[:5]
    this_code = sparse_encode_na(this_x, dict_, alpha=1)
    assert this_code.shape == (this_x.shape[0], n_components)
    
    this_x = X_na[:5]
    this_code = sparse_encode_na(this_x, dict_, alpha=1)
    assert this_code.shape == (this_x.shape[0], n_components)
        
def test_encode_reconstruction():
    
    code = sparse_encode_na(X, dict_, alpha=.0001)
    assert_array_almost_equal(np.dot(code, dict_), X, decimal=2)
    
    code = sparse_encode_na(X_na, dict_, alpha=.0001)
    X_new = np.dot(code, dict_)
    assert_array_almost_equal(X_new[Mask],
                              X[Mask], decimal=2)
       
def test_dict_learning_na():
    
    code, dict_ = dict_learning_na(X, n_components = 12, alpha = .0001)
    assert_array_almost_equal(np.dot(code, dict_), X, decimal=2)
    print(code.shape)
    print(dict_.shape)
    
    
rng_global = np.random.RandomState(0)
n_samples, n_features = 10, 8
n_components = 12
X = rng_global.randn(n_samples, n_features)
code, dict_ = dict_learning_online(X, n_components = 12, alpha = .0001)

X_na = X.copy()
X_na[0,0] = np.nan
X_na[1,1] = np.nan
Mask = np.where(~np.isnan(X_na))

test_shape_encode()
test_encode_reconstruction()
# test_dict_learning_na()